# APIs and Python

## The Many Use Cases for APIs
APIs can actually be used for many things - not just for retrieving information. Twilio has an API that allows you to write a script to call their API to send text messages to people. GitHub has an API for creating new repositories. Many services have an API allowing a computer to automate a task that a person might otherwise have to do through a website - whether uploading a photo to Flickr, searching for a company name in a state database or getting a list of garbage collection times for a municipality.

### _Learning Goals:_<br/>
  - Identify and discuss APIs
  - Discuss and explain different request (GET, POST, PUT, DELETE) and CRUD operations
  - Explore the attributes of a response object
  - Check the status of a request and interpret status codes
  - Access data from an API using the requests library
  - Create a pandas dataframe from the data returned from an API and visualize the data
  - If time, target an API of their own!!!

## Limitations of APIs
When working with APIs, there are some limitations you have to be aware of - especially relating to scope and scale.

* **Scope** - Just because a company has an API and has information, it doesn't mean you can get all of the information through their API. 

* **Scale** - Some APIs are provided for free as a public service. Others you have to pay for, or allow you to perform activities (like sending a text message) that you pay for. Make sure that you know what the rate limits are and that your use case isn't going to need more API calls than you will be able to make.

Let's take a look at the `requests` library and it's uses.

### "Requests is the only Non-GMO HTTP library for Python, safe for human consumption."

![img-req](requests-pic.jpeg)

Straight from the `requests` [documentation](https://pypi.org/project/requests/)

![nongmao](requests_med.png)

"Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor. "<br>
[More requests documentation](https://2.python-requests.org//en/v0.10.6/api/)

### Let's get started!

In [29]:
import requests

In [30]:
!pip install requests

### Types of requests

We will mostly use GET, but there are other options

Let's say you are looking at an API for a car rental company like Hertz, or maybe even Zipcar.

The following different requests will generate these different responses.

|Request        | Result        |
|-------------|-------------|
|GET /stores/   | User sees the list of stores|
|GET /rentals/  | User sees the history of car rentals|
|POST /rentals/ | User rents a car|
|PUT /rentals/{id}/| User changes destination store|
|DELETE /rentals/{id}/| User cancels the active car rental|




In [31]:
rental_json = [{
    "id": 1,
    "name": "John St",
    "location": [29, 95],
    "available_cars_quantity": 10
},
{
    "id": 2,
    "name": "Scott St.",
    "location": [29.5, 95.7],
    "available_cars_quantity": 2
}]

In [32]:
#create a post request from https://httpbin.org/post and view it in json format

r = requests.post() #put url and data = {'key':'value'} format here in the bananas!
r.json()

TypeError: post() missing 1 required positional argument: 'url'

### Request Class and Attributes

In [ ]:
#create a GET request from https://api.github.com/events and view it in json format

r = requests.get('https://api.github.com/events') #put url here in the bananas!
type(r)

Let's check the `attributes` of a `Response` object. <br>
https://2.python-requests.org//en/v0.10.6/api/

Some key attributes:

- url
- text
- status_code
- headers
- error

### Checking out the status of your request

In [ ]:
#get the status of your post request
r.status_code

#### Types of status codes
1xx - Informational responses

2xx - Success
- 200 OK
- 201 Created
- 204 No Content

3xx - Redirection

- 301 Moved Permanently (permanent URL redirection)
- 304 Not Modified (A conditional GET or HEAD request has been received and would have resulted in a 200 OK response if it were not for the fact that the condition evaluated to false.)

4xx - Client errors

- 400 Bad Request
- 401 Unauthorized
- 403 Forbidden
- 404 Not Found

5xx - Server errors

- 500 Internal Server Error

https://http.cat

In [33]:
#get status code for a "broken" link

r = requests.get('https://api.github.com/fake-ending')
r.status_code

404

Can also explictly ask for the returned format to be json as a method.

In [34]:
r.json()

{'message': 'Not Found',
 'documentation_url': 'https://developer.github.com/v3'}

#### A note on errors and exceptions with the Requests library

There are a number of exceptions and error codes you need to be familiar with when using the Requests library in Python.

- The Requests library will raise a ConnectionError exception if there is a network problem like a DNS failure, or refused connection.
- These are rare, but with invalid HTTP responses, Requests will also raise an HTTPError exception. 
- A Timeout exception will be raised if a request times out.
- If and when a request exceeds the preconfigured number of maximum redirections, then a TooManyRedirects exception will be raised

Now that we know a bit more about APIs and how to interact with them in python, let's actually investigate an API from scratch. 

To start, go over to the API documentation at: 

https://dev.socrata.com/foundry/data.cityofnewyork.us/fhrw-4uyv


<img src="311_api_docs.png">

## Make an initial API call to retrieve 311 complaints from a neighborhood or zip code of your choice in New York City.

### Using Requests

In [56]:
import requests
import pandas as pd

zip_code = '11004'

# can't figure out date ranges at the moment...
start_date = '2018-01-01T12:00:00'
end_date = '2018-02-01T12:00:00'

# create pull request based on parameters
# https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9 click API

url = "https://opendata.arcgis.com/datasets/89bfd2aed9a142249225a638448a5276_29.geojson"

# do the pull
response = requests.get(url)
response.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1001,
    'ID': '4D35075',
    'TYPE': 'Multiple_Gunshots',
    'SOURCE': 'WashingtonDC4D',
    'LATITUDE': 38.933,
    'LONGITUDE': -77.003,
    'DATETIME': '2014-05-03T01:55:35.000Z'},
   'geometry': {'type': 'Point',
    'coordinates': [-77.0030000001, 38.9329999998]}},
  {'type': 'Feature',
   'properties': {'OBJECTID': 1002,
    'ID': '4D35076',
    'TYPE': 'Multiple_Gunshots',
    'SOURCE': 'WashingtonDC4D',
    'LATITUDE': 38.933,
    'LONGITUDE': -77.003,
    'DATETIME': '2014-05-03T01:55:50.000Z'},
   'geometry': {'type': 'Point',
    'coordinates': [-77.0030000001, 38.9329999998]}},
  {'type': 'Feature',
   'properties': {'OBJECTID': 1003,
    'ID': '4D35077',
    'TYPE': 'Multiple_Gunshots',
    'SOURCE': 'WashingtonDC4D',
    'LATITUDE': 38.933,
    'LONGITUDE': -77.003,
    'DATETIME': '2014-05-03T01:57:17.000Z'},
   'geometry': {'type': 'Point',
    'coordinates': [-77.0030000001

In [58]:
response.json().keys()

dict_keys(['type', 'features'])

In [51]:
from pandas.io.json import json_normalize

In [59]:
df = pd.DataFrame.from_dict(json_normalize(response.json()['features']))

In [63]:
df.columns = [X.replace('properties.', '') for X in df.columns]

In [66]:
df.columns = [X.replace('geometry.', '') for X in df.columns]

In [69]:
import cufflinks as cf

In [68]:
df.tail()

,coordinates,type,DATETIME,ID,LATITUDE,LONGITUDE,OBJECTID,SOURCE,TYPE,type
32975,"[-77.0159999994, 38.9190000003]",Point,2018-12-02T06:22:56.000Z,3D76962,38.919,-77.016,32370,WashingtonDC3D,Multiple_Gunshots,Feature
32976,"[-76.9359999995, 38.913]",Point,2018-12-02T07:52:48.000Z,6D232666,38.913,-76.936,32371,WashingtonDC6D,Multiple_Gunshots,Feature
32977,"[-77.001, 38.8279999999]",Point,2018-12-02T08:46:31.000Z,7D301678,38.828,-77.001,32372,WashingtonDC7D,Single_Gunshot,Feature
32978,"[-76.9680000001, 38.8509999996]",Point,2018-12-02T08:56:21.000Z,7D301679,38.851,-76.968,32373,WashingtonDC7D,Single_Gunshot,Feature
32979,"[-76.9439999997, 38.8919999998]",Point,2018-12-02T09:26:31.000Z,6D232667,38.892,-76.944,32374,WashingtonDC6D,Gunshot_or_Firecracker,Feature


## Briefly Explore the Structure of the Response You Received.

### Let's look at the output of our API call to NYC 311

In [14]:
print(type(data))

<class 'list'>


In [15]:
len(data)

1000

In [16]:
data[0]

{'address_type': 'ADDRESS',
 'agency': 'NYPD',
 'agency_name': 'New York City Police Department',
 'bbl': '4084460001',
 'borough': 'QUEENS',
 'city': 'GLEN OAKS',
 'closed_date': '2019-05-14T00:45:22.000',
 'community_board': '13 QUEENS',
 'complaint_type': 'Noise - Residential',
 'created_date': '2019-05-13T23:06:30.000',
 'cross_street_1': '73 AVENUE',
 'cross_street_2': 'BEND',
 'descriptor': 'Banging/Pounding',
 'due_date': '2019-05-14T07:06:30.000',
 'facility_type': 'Precinct',
 'incident_address': '73-01 260 STREET',
 'incident_zip': '11004',
 'latitude': '40.75076013157645',
 'location': {'type': 'Point',
  'coordinates': [-73.717529079357, 40.750760131576]},
 'location_type': 'Residential Building/House',
 'longitude': '-73.7175290793568',
 'open_data_channel_type': 'ONLINE',
 'park_borough': 'QUEENS',
 'park_facility_name': 'Unspecified',
 'resolution_action_updated_date': '2019-05-14T00:45:22.000',
 'resolution_description': 'The Police Department responded to the complaint

### Libraries for company-specific APIs

Google has its own libraries you can import to access their apis. <br>
NYC also uses the `sodapy` library, Socrata, to manage their open data. 


#### Extra credit for later if you want to play with it

In [12]:
# Install before running 
!pip install sodapy

#### Tokenize yourself, B!

https://dev.socrata.com/foundry/data.cityofnewyork.us/fhrw-4uyv

Scroll down and click to sign up for an app token! This is instantaneous and simple. No credit cards required!


In [ ]:
token = '' 

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", token)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("fhrw-4uyv", incident_zip = '11004', limit=1000)

In [13]:
type(results)

requests.models.Response

In [ ]:
len(results)

In [ ]:
results[0]

In [ ]:

df_1 = pd.DataFrame(results)

print(len(df_1))
print(df_1.columns)
df_1.head()

## Create a Pandas DataFrame of the Data From the Response

In [ ]:
df_2 = pd.DataFrame(data)

print(len(df_2))
print(df_2.columns)
df_2.head()

## Create a Histogram of the Complaint Types From Your Dataset

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_2.complaint_type.value_counts().plot(kind='barh', figsize=(8,12))

## If we have time:

https://github.com/toddmotto/public-apis


find a buddy, find a free api, get the key, and do a get. Try to transform the response into a df.